In [0]:
import csv
import math
import numpy as np
import pandas as pd
import time
import pickle
from scipy import stats
from sklearn import model_selection
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.model_selection import train_test_split
from scipy import interpolate
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.model_selection import KFold
np.set_printoptions(formatter={'float_kind':'{:1}'.format})

In [0]:
import keras
import os
from keras import regularizers
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.utils import to_categorical
import tensorflow as tf
np.random.seed(1000)

In [0]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
link = 'https://drive.google.com/open?id=1xgrvzI2k1Jy4chEEVnYZpAqMuiVcbKFn'
fluff, id = link.split('=')
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('images_v1.csv')  

data0 = pd.read_csv("images_v1.csv")
data_images=np.copy(data0.values)
data_images = np.delete(data_images, 0, 1)
print(data_images.shape)
print(data_images)

(48500, 1024)
[[138.0 137.0 211.0 ... 155.0 155.0 158.0]
 [159.0 149.0 161.0 ... 160.0 160.0 160.0]
 [167.0 161.0 161.0 ... 127.0 131.0 130.0]
 ...
 [247.0 244.0 246.0 ... 156.0 153.0 145.0]
 [249.0 255.0 255.0 ... 207.0 205.0 209.0]
 [116.0 117.0 118.0 ... 169.0 169.0 170.0]]


In [0]:
link = 'https://drive.google.com/open?id=1W0awd8vVT6IAu5C6Y5QUGR1zCkNLKzDu'
fluff, id = link.split('=')
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('test_images.csv')  

test_data0 = pd.read_csv("test_images.csv")
test_data_images=np.copy(test_data0.values)
test_data_images = np.delete(test_data_images, 0, 1)
print(test_data_images.shape)
print(test_data_images)

(9700, 1024)
[[106.0 93.0 59.0 ... 119.0 98.0 96.0]
 [235.0 232.0 231.0 ... 63.0 68.0 185.0]
 [154.0 150.0 151.0 ... 161.0 162.0 162.0]
 ...
 [136.0 203.0 202.0 ... 116.0 116.0 114.0]
 [128.0 25.0 31.0 ... 78.0 75.0 104.0]
 [165.0 158.0 152.0 ... 182.0 187.0 109.0]]


In [0]:
link = 'https://drive.google.com/open?id=19H0F4oOF8oLhwqn7zc2hSajnR-jdor-9'
fluff, id = link.split('=')
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('labels.csv')

labels0 = pd.read_csv("labels.csv")
labels=np.copy(labels0.values)
labels = np.delete(labels, 0, 1)
labels=labels.reshape(-1)
print(labels.shape)
print(labels)

(48500,)
[7.0 2.0 9.0 ... 7.0 3.0 7.0]


In [0]:
data_images = data_images.flatten()
print(data_images.shape)
data_images = data_images.reshape(-1,32,32,1)
print(data_images.shape)
test_data_images = test_data_images.flatten()
print(test_data_images.shape)
test_data_images = test_data_images.reshape(-1,32,32,1)
print(test_data_images.shape)
labels = to_categorical(labels)
print(labels.shape)
#data_images = data_images.reshape(data_images.shape[0], 32, 32, 1)
#test_data_images = test_data_images.reshape(test_data_images.shape[0], 32, 32, 1)
#print(test_data_images.shape)

(49664000,)
(48500, 32, 32, 1)
(9932800,)
(9700, 32, 32, 1)
(48500, 10)


In [56]:
weight_decay = 1e-4
model = Sequential()
model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay), input_shape=data_images.shape[1:]))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
 
model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))
 
model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))
 
model.add(Flatten())
model.add(Dense(10, activation='softmax'))
 
model.summary()
opt_rms = keras.optimizers.rmsprop(lr=0.001,decay=1e-6)
model.compile(loss='categorical_crossentropy', optimizer=opt_rms, metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_65 (Conv2D)           (None, 32, 32, 32)        320       
_________________________________________________________________
activation_67 (Activation)   (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_27 (Batc (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_66 (Conv2D)           (None, 32, 32, 32)        9248      
_________________________________________________________________
activation_68 (Activation)   (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_28 (Batc (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d_29 (MaxPooling (None, 16, 16, 32)        0         
__________

In [57]:
model.fit(data_images,labels,batch_size=64,epochs=70,validation_split=0.1)

Train on 43650 samples, validate on 4850 samples
Epoch 1/70
43650/43650 [==============================] - 35s 805us/step - loss: 2.1226 - acc: 0.3830 - val_loss: 2.0727 - val_acc: 0.3897
Epoch 2/70
43650/43650 [==============================] - 30s 691us/step - loss: 1.4991 - acc: 0.5325 - val_loss: 1.7133 - val_acc: 0.4757
Epoch 3/70
43650/43650 [==============================] - 27s 615us/step - loss: 1.3289 - acc: 0.5913 - val_loss: 1.4436 - val_acc: 0.5493
Epoch 4/70
43650/43650 [==============================] - 29s 662us/step - loss: 1.2360 - acc: 0.6267 - val_loss: 1.4890 - val_acc: 0.5557
Epoch 5/70
43650/43650 [==============================] - 27s 609us/step - loss: 1.1751 - acc: 0.6513 - val_loss: 1.3429 - val_acc: 0.5986
Epoch 6/70
43650/43650 [==============================] - 29s 655us/step - loss: 1.1227 - acc: 0.6714 - val_loss: 1.3323 - val_acc: 0.6171
Epoch 7/70
43650/43650 [==============================] - 26s 606us/step - loss: 1.0853 - acc: 0.6889 - val_loss: 1.5

In [59]:
predicted_images = model.predict_classes(test_data_images)
print(predicted_images.shape)
print(predicted_images)

(9700,)
[6 0 1 ... 8 4 3]


In [0]:
from google.colab import files
pd.DataFrame(predicted_images.astype(int)).to_csv("pred_v10.csv")
files.download("pred_v10.csv")

In [0]:
# 1st Convolutional Layer
model = Sequential()
model.add(Conv2D(filters=64, input_shape=data_images.shape[1:], kernel_size=(3,3), strides=(1,1), padding='same'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(1,1), padding='same'))
model.add(Activation('relu'))
# Max Pooling

# 2nd Convolutional Layer
model.add(Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), padding='same'))
model.add(Activation('relu'))
# Max Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(1,1), padding='same'))

# 3rd Convolutional Layer
model.add(Conv2D(filters=96, kernel_size=(3,3), strides=(1,1), padding='same'))
model.add(Activation('relu'))

# 4th Convolutional Layer
model.add(Conv2D(filters=96, kernel_size=(3,3), strides=(1,1), padding='same'))
model.add(Activation('relu'))

# 5th Convolutional Layer
model.add(Conv2D(filters=192, kernel_size=(3,3), strides=(1,1), padding='same'))
model.add(Activation('relu'))
# Max Pooling
#model.add(MaxPooling2D(pool_size=(2,2), strides=(1,1), padding='same'))

# Passing it to a Fully Connected layer
model.add(Flatten())
# 1st Fully Connected Layer
model.add(Dense(4096, input_shape=(32*32*1,)))
model.add(Activation('relu'))
# Add Dropout to prevent overfitting
model.add(Dropout(0.4))

# 2nd Fully Connected Layer
model.add(Dense(4096))
model.add(Activation('relu'))
# Add Dropout
model.add(Dropout(0.4))

# 3rd Fully Connected Layer
model.add(Dense(1000))
model.add(Activation('relu'))
# Add Dropout
model.add(Dropout(0.4))

# Output Layer
model.add(Dense(10))
model.add(Activation('softmax'))

model.summary()
#aa=SGD(ls=0.0005)
# Compile the model
model.compile(loss=keras.losses.categorical_crossentropy, optimizer='adam', metrics=['accuracy'])